In [ ]:
%classpath add jar ../target/jvm/uberjar/podsm-0.1.0-SNAPSHOT-standalone.jar
(clojure.lang.Compiler/loadFile "jupyter_helpers.clj")
(require '[clojure.set :refer [rename-keys]]
         '[jupyter-helpers :refer [load-experiment]]
         '[podsm.utils :refer [map-vals]])

In [ ]:
(def dataset->base-trouble-factor {"fog_release" 1
                                   "gas" 8
                                   "higgs" 4
                                   "nasa" 3
                                   "occupancy" 1
                                  "sensorless-drive" 4
                                   "gesture-master" 1
                                   "parkinson" 1
                                   "sea-gradual" 1
                                   "rbf-moderate" 1
                                   "Skin_NonSkin" 1
                                   })
(def datasets (keys dataset->base-trouble-factor))

(defn get-experiment-keys
    [dataset]
    (let [base-trouble-factor (get dataset->base-trouble-factor dataset)
          cent-exps {(keyword (str dataset "-centralised")) :centralised}
          trouble-factor-modifiers [1 1.5 2]
          podsm-exps (into {} (map #(vector (keyword (str dataset "-podsm:max-conf-tf" (* % base-trouble-factor)))
                                           (keyword (str "podsm-conf-tf" %)))
                                  trouble-factor-modifiers))]
      ;;   (println " " (merge cent-exps podsm-exps))
        (merge podsm-exps cent-exps)))

(defn load-summary-file
    [dataset file]
    (let [experiments (load-experiment (str "workspace/" dataset) file)
          cent-experiment (first (load-experiment (str "workspace/" dataset"/" "central") file))
         total-experiments (merge experiments cent-experiment)]
       ;; (println "ex" total-experiments)
       (map vector (map :label total-experiments) total-experiments) 
        )
)
(defn load-experiment-summaries
    [dataset]

                ;; :summary has repeated labels from repeated experiments, but they have the same values except for timing, so we don't mind because...
                (into {} (map-vals :full (load-summary-file dataset :summary)))
                
)
(defn rank-experiments
    [experiment-summaries rank-by]
    (->> (vals experiment-summaries)
         (map rank-by)
         (map vector (keys experiment-summaries))
         (remove #(nil? (second %)))
         (sort-by second)
         (map first)
         (map-indexed (fn [idx exp] [exp (inc idx)]))
         (into {})))

(defn get-mean-ranks
    [rank-by]
    (let [dataset-rankings
          (for [dataset datasets]
              (let [experiment-keys (get-experiment-keys dataset)]
                 ;;  (println experiment-keys)
                  (-> (load-experiment-summaries dataset)
                      (select-keys (keys experiment-keys))
                      (rename-keys experiment-keys)
                      (rank-experiments rank-by))))
          mean-ranks
          (->> dataset-rankings
               (apply merge-with +)
               (map-vals #(float (/ % (count datasets)))))
          rank-sum (reduce + (vals mean-ranks))]
       ;;  (println dataset-rankings)
        ;; Assert total of mean ranks is 4!
        ;;(when (not= "10.00000" (format "%.5f" rank-sum))
          ;;  (throw (Exception. (str "Ranks do not sum to 4! (number of experiments compared): " rank-sum))))
        mean-ranks))

In [ ]:
(import '[java.awt Color]
        '[com.twosigma.beakerx.chart.legend LegendPosition LegendLayout]
        '[com.twosigma.beakerx.chart.xychart Plot CombinedPlot]
        '[com.twosigma.beakerx.chart.xychart.plotitem Points ShapeType])

(def experiment-keys [:centralised :podsm-conf-tf1 :podsm-conf-tf1.5 :podsm-conf-tf2])
(def experiment-labels ["Centralised" "PODSM Conf (TF 1×)" "PODSM Conf (TF 1.5×)" "PODSM Conf (TF 2×)"])
(def colors [(Color/decode "#ff96a8") (Color/decode "#f55774") (Color/decode "#d12249") (Color/decode "#1b2126")])
(def outline-colors [(Color/black) (Color/black) (Color/black) (Color/black) (Color/black)])
(def shapes [ShapeType/CIRCLE ShapeType/TRIANGLE ShapeType/SQUARE ShapeType/DIAMOND])
(def sizes [10 13 10 10])

(let [
         
        comm-plot (doto (Plot.)
                   (.setLegendPosition LegendPosition/TOP)
                   (.setLegendLayout LegendLayout/HORIZONTAL)
                   (.setYLabel "Mean Comm Rank")
                   (.setXBound [1 4])
                   (.setYBound [1 4]))
     
      comb-plot (doto (CombinedPlot.)
                   
                    (.add comm-plot)
                   ;;(.setInitWidth 500)
                   ;; (.setInitHeight 350)
                    (.setXLabel "Mean Accuracy Rank"))
      accuracy-ranks (get-mean-ranks #(- 1 (:accuracy %))) ;; Order by inverse of accuracy, so lower-ranks are better.
     
     comm-ranks (get-mean-ranks :proportion-transmitted)
     ]
   ;; (println "ranks" accuracy-ranks " " mrtcp-ranks " " comm-ranks )
    (doseq [[experiment-key label outline-color color shape size]
            (map vector experiment-keys experiment-labels outline-colors colors shapes sizes)]
    
          (println "Accuracy/Comm distance from origin"
                 experiment-key (Math/sqrt (+ (* (experiment-key accuracy-ranks) (experiment-key accuracy-ranks))
                                              (* (experiment-key comm-ranks) (experiment-key comm-ranks)))))                                    
        
    
        (.add comm-plot
            (doto (Points.)
                  (.setX [(experiment-key accuracy-ranks)])
                  (.setY [(experiment-key comm-ranks)])
                  (.setShape shape)
                  (.setColor color)
                  (.setOutlineColor outline-color)
                  (.setSize size)
                  (.setDisplayName (str label))))
       
                  )
    comb-plot)